In [ ]:
#Librería para manipular datos
import pandas as pd
# llamar a la API de Zendesk
import numpy as np
import httplib2
import time
import json
import requests
import base64
import glob
import os
from os import walk

#config
pd.set_option('display.max_columns', None)
#@markdown #***INGRESO DE USUARIO***
Dominio = '' #@param {type: "string"}
Email = '' #@param {type:"string"}
Token = ''#@param {type:"string"}
start_time="" #@param {type:"string"}

z_domain= Dominio
email= Email
contraseña= Token


#@markdown
authentication=email+"/token:"+contraseña
message_bytes = authentication.encode('ascii')
base64_bytes=base64.b64encode(message_bytes).decode('ascii')

headers = {'Authorization': 'Basic '+base64_bytes, 'Content-Type': 'application/json'}

carpeta_contenedora="/content/"

In [ ]:
def data_ticket(z_domain,start_time,headers):
  url = z_domain+"/api/v2/incremental/tickets.json?start_time="   +start_time+   "&include=metric_sets,slas"
  status = 0
  while status != 200:
    r = requests.get(url, headers=headers)
    
    status = r.status_code
    if status == 200:
      print("----llamada de api exitosa: ", status)
      data=json.loads(r.text)
      is_cont=data['end_of_stream']
      end_time=data['end_time']
    else:
      print("----llamada de api fracasada: ", status)
      time.sleep(10)
  

  
  
  #print("cantidad",data['count'])

  df_ticket=pd.DataFrame(data['tickets'])

  #se traspasa las variables del metric_set
  group_stations=[]
  assignee_stations=[]
  reopens=[]
  replies=[]
  assignee_updated_at=[]
  requester_updated_at=[]
  status_updated_at=[]
  initially_assigned_at=[]
  assigned_at=[]
  solved_at=[]
  latest_comment_added_at=[]
  reply_time_in_minutes_calendar=[]
  reply_time_in_minutes_business=[]
  first_resolution_time_in_minutes_calendar=[]
  first_resolution_time_in_minutes_business=[]
  full_resolution_time_in_minutes_calendar=[]
  full_resolution_time_in_minutes_business=[]
  agent_wait_time_in_minutes_calendar=[]
  agent_wait_time_in_minutes_business=[]
  requester_wait_time_in_minutes_calendar=[]
  requester_wait_time_in_minutes_business=[]
  on_hold_time_in_minutes_calendar=[]
  on_hold_time_in_minutes_business=[]

  for index, i in enumerate(df_ticket['metric_set']):
    try:
      group_stations.append(i['group_stations'])
      assignee_stations.append(i['assignee_stations'])
      reopens.append(i['reopens'])
      replies.append(i['replies'])
      assignee_updated_at.append(i['assignee_updated_at'])
      requester_updated_at.append(i['requester_updated_at'])
      status_updated_at.append(i['status_updated_at'])
      initially_assigned_at.append(i['initially_assigned_at'])
      assigned_at.append(i['assigned_at'])
      solved_at.append(i['solved_at'])
      latest_comment_added_at.append(i['latest_comment_added_at'])
      reply_time_in_minutes_calendar.append(i['reply_time_in_minutes']['calendar'])
      reply_time_in_minutes_business.append(i['reply_time_in_minutes']['business'])
      first_resolution_time_in_minutes_calendar.append(i['first_resolution_time_in_minutes']['calendar'])
      first_resolution_time_in_minutes_business.append(i['first_resolution_time_in_minutes']['business'])
      full_resolution_time_in_minutes_calendar.append(i['full_resolution_time_in_minutes']['calendar'])
      full_resolution_time_in_minutes_business.append(i['full_resolution_time_in_minutes']['business'])
      agent_wait_time_in_minutes_calendar.append(i['agent_wait_time_in_minutes']['calendar'])
      agent_wait_time_in_minutes_business.append(i['agent_wait_time_in_minutes']['business'])
      requester_wait_time_in_minutes_calendar.append(i['requester_wait_time_in_minutes']['calendar'])
      requester_wait_time_in_minutes_business.append(i['requester_wait_time_in_minutes']['business'])
      on_hold_time_in_minutes_calendar.append(i['on_hold_time_in_minutes']['calendar'])
      on_hold_time_in_minutes_business.append(i['on_hold_time_in_minutes']['business'])
    except:
      group_stations.append(None)
      assignee_stations.append(None)
      reopens.append(None)
      replies.append(None)
      assignee_updated_at.append(None)
      requester_updated_at.append(None)
      status_updated_at.append(None)
      initially_assigned_at.append(None)
      assigned_at.append(None)
      solved_at.append(None)
      latest_comment_added_at.append(None)
      reply_time_in_minutes_calendar.append(None)
      reply_time_in_minutes_business.append(None)
      first_resolution_time_in_minutes_calendar.append(None)
      first_resolution_time_in_minutes_business.append(None)
      full_resolution_time_in_minutes_calendar.append(None)
      full_resolution_time_in_minutes_business.append(None)
      agent_wait_time_in_minutes_calendar.append(None)
      agent_wait_time_in_minutes_business.append(None)
      requester_wait_time_in_minutes_calendar.append(None)
      requester_wait_time_in_minutes_business.append(None)
      on_hold_time_in_minutes_calendar.append(None)
      on_hold_time_in_minutes_business.append(None)

  df_ticket['grupos asignados']=group_stations
  df_ticket['agentes asignados']=assignee_stations
  df_ticket['reaperturas']=reopens
  df_ticket['cantidad de respuestas']=replies
  df_ticket['ultima actualizacion agente']=assignee_updated_at
  df_ticket['ultima actualizacion solicitante']=requester_updated_at
  df_ticket['ultima actualizacion estado']=status_updated_at
  df_ticket['inicialmente asignado']=initially_assigned_at
  df_ticket['fecha asignacion']=assigned_at
  df_ticket['ultimo comentario agregado']=latest_comment_added_at
  df_ticket['tiempo respuesta calendario']=reply_time_in_minutes_calendar
  df_ticket['tiempo respuesta negocio']=reply_time_in_minutes_business
  df_ticket['primera resolucion calendario']=first_resolution_time_in_minutes_calendar
  df_ticket['primera resolucion negocio']=first_resolution_time_in_minutes_business
  df_ticket['resolucion completa calendario']=full_resolution_time_in_minutes_calendar
  df_ticket['resolucion completa negocio']=full_resolution_time_in_minutes_business
  df_ticket['espera agente calendario']=agent_wait_time_in_minutes_calendar
  df_ticket['espera agente negocio']=agent_wait_time_in_minutes_business
  df_ticket['espera solicitante calendario']=requester_wait_time_in_minutes_calendar
  df_ticket['espera solicitante negocio']=requester_wait_time_in_minutes_business
  df_ticket['tiempo espera calendario']=on_hold_time_in_minutes_calendar
  df_ticket['tiempo espera negocio']=on_hold_time_in_minutes_business
  #time.sleep(6)
  ## SLA tratamiento
  frt_time_status=[]
  frt_time_brkn=[]
  frt_time_day_brkn=[]
  frt_time_day_brkn_min=[]
  agnt_time_status=[]
  agnt_time_brkn=[]
  agnt_time_day_brkn=[]
  agnt_time_day_brkn_min=[]
  count_ewq=0
  for ind in df_ticket.index:
    ad1=None
    ad2=None
    ad3=None
    ad4=None
    bd1=None
    bd2=None
    bd3=None
    bd4=None
    for x in df_ticket['slas'][ind]['policy_metrics']:
      if x['metric']=="first_reply_time":
        if x['breach_at'] is None:
          ad1=x['stage']
          ad2=True
          ad3=x['breach_at']
          ad4=None
          break
        else:
          ad1=x['stage']
          ad2=False
          ad3=x['breach_at']
          if 'days' in x:
            ad4=x['days']*1440
          if 'hours' in x:
            ad4=x['hours']*60
          if 'minutes' in x:
            ad4=x['minutes']
          break
    for x in df_ticket['slas'][ind]['policy_metrics']:
      if x['metric']=="agent_work_time":
        if x['breach_at'] is None:
          bd1=x['stage']
          bd2=True
          bd3=x['breach_at']
          bd4=None
          break
        else:
          bd1=x['stage']
          bd2=False
          bd3=x['breach_at']
          if 'days' in x:
            bd4=x['days']*1440
          if 'hours' in x:
            bd4=x['hours']*60
          if 'minutes' in x:
            bd4=x['minutes']
          break  
    frt_time_status.append(ad1)
    frt_time_brkn.append(ad2)
    frt_time_day_brkn.append(ad3)
    frt_time_day_brkn_min.append(ad4)
    agnt_time_status.append(bd1)
    agnt_time_brkn.append(bd2)
    agnt_time_day_brkn.append(bd3)
    agnt_time_day_brkn_min.append(bd4)
      


  
  df_ticket['SLA FIRST RESPONSE - Estado']=frt_time_status
  df_ticket['SLA FIRST RESPONSE - ¿Roto?']=frt_time_brkn
  df_ticket['SLA FIRST RESPONSE - Fecha roto']=frt_time_day_brkn
  df_ticket['SLA FIRST RESPONSE - Tiempo roto']=frt_time_day_brkn_min
  df_ticket['SLA AGENT RESPONSE - Estado']=agnt_time_status
  df_ticket['SLA AGENT RESPONSE - ¿Roto?']=agnt_time_brkn
  df_ticket['SLA AGENT RESPONSE - Fecha roto']=agnt_time_day_brkn
  df_ticket['SLA AGENT RESPONSE - Tiempo roto']=agnt_time_day_brkn_min

  return df_ticket, is_cont, end_time

In [ ]:
def data_user(z_domain,start_time,headers):
  continua2=True
  continua=True
  org_time=str(946684800)
  user_time=str(946684800)
  all_org={}
  iter=1
  print("---Lo lamentamos, esto puede demorar")
  while continua2:
    print("---extración de info org iter "+str(iter)+",COD "+org_time)
    url2 = z_domain+"/api/v2/incremental/organizations.json?start_time="+org_time
    http = httplib2.Http()
    response, content = http.request(url2, 'GET', headers=headers)
    data2=json.loads(content)
    time.sleep(6)
    #print(data2)
    #print("_________")

    if(data2['end_of_stream']):
      continua2=False 
    org_time=str(data2['end_time'])
  print("---Se obtuvo la información de las organizaciones")
  #__________________________________________________________________________________________________
  all_user={}
  iter=1
  while continua:
    print("---extración de info usuario iter "+str(iter)+",COD "+user_time)
    url = z_domain+"/api/v2/incremental/users.json?start_time="+user_time
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(6)
    #print(data)
    #print("_________")
    for idx, x in enumerate(data['users']):
      if x['organization_id'] is not None:
        nom_org=all_org[str(x['organization_id'])][0]
        hh_mm=all_org[str(x['organization_id'])][1]
        hh_tt=all_org[str(x['organization_id'])][2]
      else:
        nom_org=""
        hh_mm=0
        hh_tt=0
      all_user[str(x['id'])]=[x['name'],x['email'],str(nom_org),hh_mm,hh_tt]

    if(data['end_of_stream']):
      continua=False 
    user_time=str(data['end_time'])
    iter=iter+1
  print("---Se obtuvo la información de los usuarios")

  return all_user


In [ ]:
def generarDataSetCamposTicket(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/ticket_fields"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    campos_de_ticket={}
    for idx, x in enumerate(data['ticket_fields']):
        id =  data['ticket_fields'][idx]['id']
        nombreCampo =  data['ticket_fields'][idx]['title']
        type =  data['ticket_fields'][idx]['type']


        if type == 'tagger' or type == 'multiselect':
            custom_field_options = data['ticket_fields'][idx]['custom_field_options']
            opciones_campo = []
            for idy, y in enumerate(custom_field_options):
                opciones_campo.append([custom_field_options[idy]['raw_name'], custom_field_options[idy]['value']])

            campos_de_ticket[id]=[nombreCampo, type, opciones_campo]

        if type == 'checkbox':
            tag = custom_field_options = data['ticket_fields'][idx]['tag']
            campos_de_ticket[id]=[nombreCampo, type, tag]

        if type != 'checkbox' and type != 'tagger' and type != 'multiselect':
            campos_de_ticket[id]=[nombreCampo, type]

    
    return campos_de_ticket

In [ ]:
def generarDataSetFormularios(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/ticket_forms"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(4)
    formularios={}
    for idx, x in enumerate(data['ticket_forms']):
        id = data['ticket_forms'][idx]['id'];
        nombre = data['ticket_forms'][idx]['name'];
        formularios[id] = nombre
        
    return formularios

In [ ]:
def generarDataSetAgentes():    
    url = z_domain+"/api/v2/users?role=agent&role=admin"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)

    agentes={}
    for idx, x in enumerate(data['users']):
        id = data['users'][idx]['id'];
        nombre = data['users'][idx]['name'];
        agentes[id] = nombre
        
    return agentes

In [ ]:
def generarDataSetMarcas(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/brands"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(4)
    marcas={}
    for idx, x in enumerate(data['brands']):
        id = data['brands'][idx]['id'];
        nombre = data['brands'][idx]['name'];
        marcas[id] = nombre
        
    return marcas

In [ ]:
def generarDataGrupo(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/groups"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(4)
    grupo={}
    for idx, x in enumerate(data['groups']):
        id = x['id']
        nombre = x['name']
        grupo[id] = nombre
        
    return grupo

In [ ]:
def etl_ticket(z_domain,start_time,headers,carpeta_contenedora):
  cont=True
  list_csv=[]
  N_1=1
  print("Estamos extrayendo las referencias para este ETL")
  #print("-Descargando data formularios")
  #lista_formulario=generarDataSetFormularios(z_domain,start_time,headers)
  #print("--Cantidad",len(lista_formulario))
  #print("-Descargando data usuarios")
  #lista_usuario=data_user(z_domain,start_time,headers)
  #print("--Cantidad",len(lista_usuario))
  #print("-Descargando data campos personalizados")
  #lista_campos=generarDataSetCamposTicket(z_domain,start_time,headers)
  #print("--Cantidad de campos",len(lista_campos))
  #print("-Descargando lista de marcas")
  #lista_marca=generarDataSetMarcas(z_domain,start_time,headers)
  #print("--Cantidad",len(lista_marca))
  #print("-Descargando lista de grupos")
  #lista_grupo=generarDataGrupo(z_domain,start_time,headers)
  #print("--Cantidad",len(lista_grupo))
  try:
    os.remove(carpeta_contenedora+"/data.csv")
  except:
    print("No se encontro un archivo data.csv")
  print("Parte extracción de los datos de tickets")
  while cont:
    print("_______________________________________________________")
    print("-Iteración "+str(N_1)+":",start_time, "Trabajando 😔")
    data,cont_1,new_start=data_ticket(z_domain,start_time,headers)
    print("--Se consiguío los datos de Zendesk")
    
    time.sleep(4)
    # Información de los usuarios
  
   

    #Se elimina las columnas innecesarias
    print("--Estoy eliminando columnas innecesarias 😩")
    #data=data.drop(columns=['via',"forum_topic_id","custom_fields","fields","index","slas"])
    #Estoy actualizando los nombres de las columnas
    print("--Cambio el nombre de las columnas 🤤")
    #data.rename(columns = {"type":"tipo","subject":"asunto","description":"descripcion","priority":"prioridad","status":"estado","recipient":"casilla receptora","requester_id":"id solicitante","submitter_id":"id remitente","assignee_id":"id agente asignado","organization_id":"id organizacion solicitante","group_id":"id grupo asignado","ticket_form_id":"id formulario"}, inplace = True)
    #Crear archivo
    data.to_csv(carpeta_contenedora+'/'+start_time+".csv")
    list_csv.append(start_time+".csv")
    
    if(cont_1):
      cont=False
    start_time=str(new_start)
    N_1=N_1+1
    print("--Lista la transformación 😉")

  
  os.chdir(carpeta_contenedora)
  file_extension = '.csv'
  all_filenames = [i for i in glob.glob(f"*{file_extension}")]
  combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])


  
  combined_csv_data = combined_csv_data.loc[:, ~combined_csv_data.columns.str.startswith('Unnamed')]
  combined_csv_data.to_csv("data.csv")
  for file in list_csv:
   os.remove(carpeta_contenedora+"/"+file)
  print("Ya se creo el archivo data_ticket_zerviz.csv 🤤")

  return combined_csv_data
  



In [ ]:
df_etl=etl_ticket(z_domain,start_time,headers,carpeta_contenedora)



Estamos extrayendo las referencias para este ETL
No se encontro un archivo data.csv
Parte extracción de los datos de tickets
_______________________________________________________
-Iteración 1: 1672531200 Trabajando 😔
----llamada de api exitosa:  200
--Se consiguío los datos de Zendesk
--Estoy eliminando columnas innecesarias 😩
--Cambio el nombre de las columnas 🤤
--Lista la transformación 😉
Ya se creo el archivo data_ticket_zerviz.csv 🤤


In [ ]:
df_etl.to_excel("archivo.xlsx")